In [ ]:
!pip install tesseract pytesseract opencv-python tesserocr  python-Levenshtein gspread oauth2client

In [12]:
from PIL import Image
import cv2
import pytesseract
from pathlib import Path
import glob
import numpy as np
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials

pytesseract.pytesseract.tesseract_cmd = ".\\tesseract\\tesseract.exe"

In [133]:
CONFIG_PATH = "C:\\Users\\Azazel\\Desktop\\projekte\\fallout\\config.json"
with open(CONFIG_PATH, encoding="utf8") as f:
    j = json.load(f)
    sheet_name = j["google_sheet_name"]
    IMG_FOLDER_PATH = j["screenshot_folder_path"]
    google_api_file_path = j["google_api_file_path"]
    effects = j["effects"]
    weapons = j["weapons"]
    effect_mod_type_order = { t:i for i,t in enumerate(j["effect_mod_type_order"])}
    word_file_path = j["word_file_path"]
    sheet_distance_between_ranged_and_melee = j["sheet_distance_between_ranged_and_melee"]
images = [x for x in glob.glob(IMG_FOLDER_PATH + "*.png")]

In [102]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(google_api_file_path, scope)
client = gspread.authorize(creds)
sheet = client.open(sheet_name).sheet1
ids = sheet.col_values(1)[1:]

In [28]:
def expand_image(img):
    old_size = img.size
    new_size = (100, 100)
    new_im = Image.new("RGB", new_size, (255, 255, 255))
    new_im.paste(img, (int((new_size[0]-old_size[0])/2),
                          int((new_size[1]-old_size[1])/2)))
    return new_im

In [29]:
def read_int(img, fl=False):
    img = expand_image(img)
    return pytesseract.image_to_string(img, lang = 'deu', config='--psm 6 -c tessedit_char_whitelist=0123456789' + (".," if fl  else "")).strip()

In [30]:
from matplotlib.pyplot import imshow
import numpy as np
def read_statistics(cropped_img):
    level = cropped_img.crop((70,40,110,85))
    dmg = cropped_img.crop((236,110,290,160))
    firerate = cropped_img.crop((210,180,290,230))
    _range = cropped_img.crop((210,220,290,260))
    precision = cropped_img.crop((236,260,285,295))
    weight = cropped_img.crop((215,290,290,340))
    level = read_int(level)
    dmg = read_int(dmg)
    firerate = read_int(firerate)
    _range =read_int(_range)
    precision = read_int(precision)
    #weight =  read_int(weight)
    #weight = float(int(weight.replace(" ", ""))/10)
    return level,dmg,firerate,_range,precision,weight


In [31]:
def read_sentence(crop):
    return pytesseract.image_to_string(crop, lang = 'deu', config=f"-c load_system_dawg=false -c load_freq_dawg=false --user-words {word_file_path}")

In [32]:
def read_name(cropped_image):
    name_as_pixels = np.array(cropped_image)
    x,y, c = name_as_pixels.shape
    for i in range(0,x):
        for j in range(0,y):
            pixel = name_as_pixels[i][j]
            diff = np.absolute(pixel - [244, 202,  91])
            if np.all(diff < 5):
                name_as_pixels[i][j] = [0,0,0]
            else:
                name_as_pixels[i][j] = [255,255,255]
    clean_name = Image.fromarray(name_as_pixels)
    line = read_sentence(clean_name)
    for name,attributes in list(weapons.items()):
        if name in line:
            return attributes["translation"], attributes["ranged"]
    return "not recognized", False

In [87]:
from Levenshtein import distance
from tqdm import tqdm_notebook as tqdm
import PIL.ImageOps 
results = []
for image in tqdm(images):
    _id = Path(image).stem.replace("ScreenShot", "")
    img = Image.open(image)
    name = img.crop((500,40,1330,85))
    thresh = 225
    fn = lambda x : 255 if x > thresh else 0
    img = img.convert('L').point(fn, mode='1')
    img = PIL.ImageOps.invert(img.convert('L'))
    statistics = img.crop((310,520,600,930))
    level = read_statistics(statistics)[0]
    weapon_type,ranged = read_name(name)
    effects1 = img.crop((520,75,1360,115))
    effects2 = img.crop((520,110,1360,145))
    effects3 = img.crop((520,145,1360,180))
    sentences = [read_sentence(effects1),read_sentence(effects2),read_sentence(effects3)]
    sentences = [s for s in sentences if len(s.strip()) > 0]
    results.append((_id,level, weapon_type, ranged, sentences))

In [89]:
out = []
for (_id,level, weapon_type, ranged,sentences) in results:
    weapon_effects = []
    for s in sentences:
        for text, attributes in list(effects.items()):
            edit_dist = distance(text, s)
            if edit_dist < 5:
                weapon_effects.append((attributes["translation"], attributes["mod_type"]))
    out.append((_id,level,weapon_type, ranged, weapon_effects))

In [94]:
def sort_effects(w_effects):
    w_effects = [(text, mod_type, effect_mod_type_order[mod_type]) for text,mod_type in w_effects]
    w_effects= sorted(w_effects, key=lambda element: (element[2]))
    return [text for text, mod, order in w_effects]

out = [(_id,level, weapon_type, ranged, sort_effects(weapon_effects)) for _id,level, weapon_type, ranged, weapon_effects in out]

In [98]:
out = sorted(cpy,key = lambda e: e[4][0])

In [100]:
ranged = [(_id,level,weapon_type, weapon_effects) for (_id,level,weapon_type, ranged, weapon_effects) in out if ranged]
melee = [(_id,level,weapon_type, weapon_effects) for (_id,level,weapon_type, ranged, weapon_effects) in out if not ranged]

[('47',
  '45',
  'Minigun',
  ['25% increased fire rate', '50 Damage resistance while aiming']),
 ('32',
  '45',
  'Ultracite gatling laser',
  ['Anti-Armor', '25% increased fire rate', '15% faster reload']),
 ('36',
  '50',
  'Gatling plasma',
  ['Anti-Armor', '25% increased fire rate', '1 Perception']),
 ('37', '50', 'LMG', ['Anti-Armor', 'Explosive']),
 ('38', '50', 'LMG', ['Anti-Armor', '25% increased fire rate']),
 ('39', '45', 'Pumpgun', ['Anti-Armor', 'Explosive']),
 ('40',
  '45',
  'Gamma gun',
  ['Bloodied', 'VATS critical shots do +50% damage', '1 Perception']),
 ('41',
  '45',
  'Gamma gun',
  ['Bloodied',
   'VATS critical shots do +50% damage',
   'Your VATS critical meter fills 15% faster']),
 ('42',
  '45',
  'Laser rifle',
  ['Bloodied', '25% increased fire rate', '1 Agility']),
 ('48',
  '45',
  'Lever action rifle',
  ['Explosive', '25% less VATS Action Point cost']),
 ('50', '50', 'Gatling gun', ['Furious', 'Explosive']),
 ('44',
  '50',
  'Combat rifle',
  ["Junki

In [143]:
def delete_rows(range_start, melee_start):
    for i in range(melee_start, melee_start + ( len(sheet_values) - melee_start) + 1):
        sheet.delete_row(melee_start)
    for i in range(range_start, melee_start -2 - sheet_distance_between_ranged_and_melee):
        sheet.delete_row(range_start)

In [148]:
sheet_values = sheet.get_all_values()
ranged_start = 3
melee_start = ranged_start + sheet_distance_between_ranged_and_melee + 2
for i, row in enumerate(sheet_values, 1):
    if row[0] == "Ranged":
        ranged_start = i + 2
    if row[0] == "Melee":
        melee_start = i + 2
delete_rows(ranged_start, melee_start)
ranged_start = 3
for i, (_id,level,weapon_type, weapon_effects) in enumerate(ranged):
    row = [weapon_effects[0], weapon_effects[1] if len(weapon_effects) > 1 else "",  \
          weapon_effects[2] if len(weapon_effects) > 2 else "", weapon_type, level, "", "", _id]
    row = [str(c) for c in row]
    sheet.insert_row(row, ranged_start + i)
    
melee_start = ranged_start + sheet_distance_between_ranged_and_melee + 2 + len(ranged)

for _id,level,weapon_type, weapon_effects in melee:
    row = [weapon_effects[0], weapon_effects[1] if len(weapon_effects) > 1 else "",  \
          weapon_effects[2] if len(weapon_effects) > 2 else "", weapon_type, level, "", "", _id]
    row = [str(c) for c in row]
    sheet.insert_row(row, melee_start)
    melee_start +=1

# TODO
* Ids aus sheet auslesen und bei exists nicht erneut lesen
* Berechnung parallelisieren
